In [10]:
from langchain.document_loaders import UnstructuredPDFLoader, PyMuPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
import os

In [11]:
api_key = os.getenv("OPENAI_API_KEY_NUS")
memory = MemorySaver()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=api_key)

In [12]:
# Load the PDF document
loader = UnstructuredPDFLoader(file_path="./data/CERT_RAG.pdf")
docs = loader.load()

# Split the document into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [18]:
# Create vector store from the document chunks
vectorstore = FAISS.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

# Set up the retriever
retriever = vectorstore.as_retriever(search_type="similarity")

In [19]:
### Build retriever tool ###
tool = create_retriever_tool(
    retriever,
    "pdf_retriever",
    "Searches and returns excerpts from the uploaded PDF document.",
)
tools = [tool]

agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [22]:
query = "What is one element teams preset during the draft plan review? (a) ART PI risks (b) Planned Business Value (c) Milestones (d) Iteration Goals"

In [23]:
from langchain_core.messages import AIMessage, HumanMessage

config = {"configurable": {"thread_id": "abc123"}}

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is one element teams preset during the draft plan review? (a) ART PI risks (b) Planned Business Value (c) Milestones (d) Iteration Goals
================================== Ai Message ==================================

One element teams present during the draft plan review is (b) Planned Business Value.
